In [ ]:
%pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
model = DiscreteBayesianNetwork([('Prize', 'Host'), ('Choice', 'Host')])

In [ ]:
cpd_prize = TabularCPD(variable='Prize', variable_card=3,
                       values=[[1/3], [1/3], [1/3]],
                       state_names={'Prize': [1, 2, 3]})

In [ ]:
cpd_choice = TabularCPD(variable='Choice', variable_card=3,
                        values=[[1/3], [1/3], [1/3]],
                        state_names={'Choice': [1, 2, 3]})

In [ ]:
host_values = np.zeros((3, 9))

In [ ]:
host_doors = [1, 2, 3]
for i, prize in enumerate(host_doors):
    for j, choice in enumerate(host_doors):
        valid = [door for door in host_doors if door != prize and door != choice]
        for v in valid:
            host_values[v-1][3*i + j] = 1 / len(valid)

cpd_host = TabularCPD(variable='Host', variable_card=3,
                      values=host_values,
                      evidence=['Prize', 'Choice'],
                      evidence_card=[3, 3],
                      state_names={'Host': [1, 2, 3], 'Prize': [1, 2, 3], 'Choice': [1, 2, 3]})

In [ ]:
model.add_cpds(cpd_prize, cpd_choice, cpd_host)
model.check_model()

True

In [ ]:
inference = VariableElimination(model)

In [ ]:
query_result = inference.query(variables=['Prize'],
                               evidence={'Choice': 1, 'Host': 3})

In [ ]:
print("P(Prize | Choice=1, Host=3):")
print(query_result)

P(Prize | Choice=1, Host=3):
+----------+--------------+
| Prize    |   phi(Prize) |
+==========+==============+
| Prize(1) |       0.3333 |
+----------+--------------+
| Prize(2) |       0.6667 |
+----------+--------------+
| Prize(3) |       0.0000 |
+----------+--------------+
